# Звіт з самостійної роботи: Алгоритми на зважених графах

## 1. Завдання та візуалізація графа

### Заданий граф (Варіант 5):
Зважений граф: `[(1,2,10), (1,3,15), (1,4,20), (2,3,35), (2,4,25), (3,4,30)]`

### Код для візуалізації графа:

```python
import networkx as nx
import matplotlib.pyplot as plt

# Задання графа
weighted_edges = [(1, 2, 10), (1, 3, 15), (1, 4, 20), (2, 3, 35), (2, 4, 25), (3, 4, 30)]
G = nx.Graph()
G.add_weighted_edges_from(weighted_edges)

print(f"Кількість вершин у графі: {G.number_of_nodes()}")
print(f"Кількість ребер у графі: {G.number_of_edges()}")

# Візуалізація
plt.figure(figsize=(10, 8))
pos = nx.spring_layout(G, seed=42)

nx.draw_nodes(G, pos, node_color='lightblue', node_size=3000, alpha=0.9)
nx.draw_edges(G, pos, width=2, alpha=0.7, edge_color='gray')
nx.draw_labels(G, pos, font_size=12, font_weight='bold')

edge_labels = nx.get_edge_attributes(G, 'weight')
nx.draw_edge_labels(G, pos, edge_labels=edge_labels, font_color='red', font_size=10)

plt.title("Візуалізація заданого зваженого графа")
plt.axis('off')
plt.show()

import networkx as nx
import matplotlib.pyplot as plt
import heapq
from collections import deque

# Задання графа
weighted_edges = [(1, 2, 10), (1, 3, 15), (1, 4, 20), (2, 3, 35), (2, 4, 25), (3, 4, 30)]
G = nx.Graph()
G.add_weighted_edges_from(weighted_edges)

print("--- Реалізація та результати алгоритмів ---")

# --- Алгоритм Дейкстри ---
print("\n=== Алгоритм Дейкстри ===")

def dijkstra(graph, start_node):
    distances = {node: float('inf') for node in graph.nodes()}
    distances[start_node] = 0
    priority_queue = [(0, start_node)]
    previous_nodes = {node: None for node in graph.nodes()}

    while priority_queue:
        current_distance, current_node = heapq.heappop(priority_queue)
        if current_distance > distances[current_node]: continue
        for neighbor in graph.neighbors(current_node):
            weight = graph[current_node][neighbor]['weight']
            distance = current_distance + weight
            if distance < distances[neighbor]:
                distances[neighbor] = distance
                previous_nodes[neighbor] = current_node
                heapq.heappush(priority_queue, (distance, neighbor))
    return distances, previous_nodes

def reconstruct_path_dijkstra(previous_nodes, start_node, end_node):
    path = deque()
    current = end_node
    while current is not None:
        path.appendleft(current)
        if current == start_node: break
        current = previous_nodes[current]
    return list(path) if path and path[0] == start_node else []

for start_node_dijkstra in G.nodes():
    distances_d, previous_nodes_d = dijkstra(G, start_node_dijkstra)
    print(f"\nРезультати зі стартової вершини {start_node_dijkstra}:")
    for target_node, dist in distances_d.items():
        path_d = reconstruct_path_dijkstra(previous_nodes_d, start_node_dijkstra, target_node)
        print(f"  До {target_node}: відстань = {dist}, шлях = {path_d}")


# --- Алгоритм Флойда-Уоршелла ---
print("\n=== Алгоритм Флойда-Уоршелла ===")

def floyd_warshall(graph):
    nodes = sorted(list(graph.nodes()))
    num_nodes = len(nodes)
    node_to_index = {node: i for i, node in enumerate(nodes)}
    index_to_node = {i: node for i, node in enumerate(nodes)}
    
    distances = [[float('inf')] * num_nodes for _ in range(num_nodes)]
    next_node = [[None] * num_nodes for _ in range(num_nodes)]

    for i in range(num_nodes): distances[i][i] = 0
    for u, v, data in graph.edges(data=True):
        u_idx, v_idx = node_to_index[u], node_to_index[v]
        weight = data['weight']
        distances[u_idx][v_idx] = weight
        distances[v_idx][u_idx] = weight
        next_node[u_idx][v_idx] = v
        next_node[v_idx][u_idx] = u

    for k_idx, k in enumerate(nodes):
        for i_idx, i in enumerate(nodes):
            for j_idx, j in enumerate(nodes):
                if distances[i_idx][j_idx] > distances[i_idx][k_idx] + distances[k_idx][j_idx]:
                    distances[i_idx][j_idx] = distances[i_idx][k_idx] + distances[k_idx][j_idx]
                    next_node[i_idx][j_idx] = next_node[i_idx][k_idx]
    return distances, next_node, nodes, node_to_index, index_to_node

def reconstruct_path_fw(start_node, end_node, next_node_matrix, index_to_node, node_to_index):
    start_idx = node_to_index[start_node]
    end_idx = node_to_index[end_node]
    if next_node_matrix[start_idx][end_idx] is None and start_idx != end_idx: return []
    path = [start_node]
    current = start_node
    while current != end_node:
        current_idx = node_to_index[current]
        next_in_path = next_node_matrix[current_idx][end_idx]
        if next_in_path is None: return []
        path.append(next_in_path)
        current = next_in_path
    return path

fw_distances_matrix, fw_next_node_matrix, fw_nodes, fw_node_to_index, fw_index_to_node = floyd_warshall(G)

print("\nМатриця найкоротших відстаней (Флойд-Уоршелл):")
header = "      " + " ".join([f"{node:^5}" for node in fw_nodes])
print(header)
print("     " + "-" * len(header))
for i, row in enumerate(fw_distances_matrix):
    print(f"{fw_nodes[i]:^5}| " + " ".join([f"{dist:^5.0f}" if dist != float('inf') else f"{'inf':^5}" for dist in row]))

print("\nПриклади шляхів (Флойд-Уоршелл):")
start_fw = 1; end_fw = 4
path_fw = reconstruct_path_fw(start_fw, end_fw, fw_next_node_matrix, fw_index_to_node, fw_node_to_index)
distance_fw = fw_distances_matrix[fw_node_to_index[start_fw]][fw_node_to_index[end_fw]]
print(f"  Шлях від {start_fw} до {end_fw}: {path_fw}, відстань: {distance_fw}")


# --- Алгоритм Крускала ---
print("\n=== Алгоритм Крускала (для Мінімального Остовного Дерева) ===")

def kruskal(graph):
    edges = sorted([(data['weight'], u, v) for u, v, data in graph.edges(data=True)])
    mst = []
    parent = {node: node for node in graph.nodes()}
    rank = {node: 0 for node in graph.nodes()}

    def find(node):
        if parent[node] == node: return node
        parent[node] = find(parent[node]) 
        return parent[node]

    def union(node1, node2):
        root1, root2 = find(node1), find(node2)
        if root1 != root2:
            if rank[root1] < rank[root2]: parent[root1] = root2
            elif rank[root1] > rank[root2]: parent[root2] = root1
            else: parent[root2] = root1; rank[root1] += 1
            return True
        return False

    for weight, u, v in edges:
        if union(u, v): mst.append((u, v, weight))
    return mst

mst_kruskal = kruskal(G)
total_weight_kruskal = sum(edge[2] for edge in mst_kruskal)

print("\nМінімальне остовне дерево (Крускал):")
for u, v, weight in mst_kruskal: print(f"  Ребро: ({u}, {v}), Вага: {weight}")
print(f"Загальна вага MST (Крускал): {total_weight_kruskal}")

mst_graph_kruskal = nx.Graph()
mst_graph_kruskal.add_weighted_edges_from(mst_kruskal)
plt.figure(figsize=(8, 6))
pos_mst = nx.spring_layout(mst_graph_kruskal, seed=42)
nx.draw_nodes(mst_graph_kruskal, pos_mst, node_color='lightgreen', node_size=2000, alpha=0.9)
nx.draw_edges(mst_graph_kruskal, pos_mst, width=2, alpha=0.7, edge_color='darkgreen')
nx.draw_labels(mst_graph_kruskal, pos_mst, font_size=12, font_weight='bold')
edge_labels_mst = nx.get_edge_attributes(mst_graph_kruskal, 'weight')
nx.draw_edge_labels(mst_graph_kruskal, pos_mst, edge_labels=edge_labels_mst, font_color='red', font_size=10)
plt.title("Мінімальне остовне дерево (Алгоритм Крускала)")
plt.axis('off')
plt.show()


# --- Алгоритм Прима ---
print("\n=== Алгоритм Прима (для Мінімального Остовного Дерева) ===")

def prim(graph, start_node):
    min_spanning_tree_edges = []
    visited = {start_node}
    priority_queue = []
    
    for neighbor, data in graph[start_node].items():
        heapq.heappush(priority_queue, (data['weight'], start_node, neighbor))

    while priority_queue:
        weight, u, v = heapq.heappop(priority_queue)
        if v in visited: continue
            
        visited.add(v)
        min_spanning_tree_edges.append((u, v, weight))
        
        for neighbor_of_v, data_of_v in graph[v].items():
            if neighbor_of_v not in visited:
                heapq.heappush(priority_queue, (data_of_v['weight'], v, neighbor_of_v))
    return min_spanning_tree_edges

start_node_prim = 1 
mst_prim = prim(G, start_node_prim)
total_weight_prim = sum(edge[2] for edge in mst_prim)

print(f"\nМінімальне остовне дерево (Прима) зі стартової вершини {start_node_prim}:")
for u, v, weight in mst_prim: print(f"  Ребро: ({u}, {v}), Вага: {weight}")
print(f"Загальна вага MST (Прима): {total_weight_prim}")

mst_graph_prim = nx.Graph()
mst_graph_prim.add_weighted_edges_from(mst_prim)
plt.figure(figsize=(8, 6))
pos_mst_prim = nx.spring_layout(mst_graph_prim, seed=42)
nx.draw_nodes(mst_graph_prim, pos_mst_prim, node_color='lightcoral', node_size=2000, alpha=0.9)
nx.draw_edges(mst_graph_prim, pos_mst_prim, width=2, alpha=0.7, edge_color='darkred')
nx.draw_labels(mst_graph_prim, pos_mst_prim, font_size=12, font_weight='bold')
edge_labels_mst_prim = nx.get_edge_attributes(mst_graph_prim, 'weight')
nx.draw_edge_labels(mst_graph_prim, pos_mst_prim, edge_labels=edge_labels_mst_prim, font_color='blue', font_size=10)
plt.title("Мінімальне остовне дерево (Алгоритм Прима)")
plt.axis('off')
plt.show()

# Відповіді на теоретичні запитання з теорії графів

## 1. Що таке граф у термінах теорії графів? Наведіть приклади реальних ситуацій, де можна застосовувати графи.

**Граф** у термінах теорії графів — це абстрактна структура, що складається з множини **вершин** (або вузлів) та множини **ребер** (або зв'язків), що з'єднують ці вершини. Граф позначається як $G = (V, E)$, де $V$ — це множина вершин, а $E$ — множина ребер.

**Приклади реальних ситуацій, де застосовуються графи:**

* **Соціальні мережі:** Вершини — це користувачі, ребра — дружні зв'язки. Дозволяє аналізувати зв'язки, знаходити спільноти, поширювати інформацію.
* **Дорожні мережі:** Вершини — це перехрестя або міста, ребра — дороги. Використовується для побудови маршрутів, навігації, оптимізації трафіку.
* **Комп'ютерні мережі (Інтернет):** Вершини — це комп'ютери або сервери, ребра — мережеві з'єднання. Застосовується для маршрутизації даних, виявлення вразливостей.
* **Системи водопостачання/газопостачання:** Вершини — насосні станції, вузли розподілу, ребра — труби. Допомагає оптимізувати потік, виявляти несправності.
* **Проєкти та залежності завдань:** Вершини — завдання проєкту, ребра — залежності між завданнями (одне завдання має бути виконане до початку іншого). Використовується для планування та управління проєктами (наприклад, діаграми Ганта).
* **Хімія (молекулярні структури):** Вершини — атоми, ребра — хімічні зв'язки. Дозволяє моделювати молекули та їх взаємодію.
* **Мережі метро/залізниць:** Вершини — станції, ребра — ділянки колії. Використовується для планування маршрутів та оптимізації руху потягів.

## 2. Які основні види графів існують? Наведіть відмінності між орієнтованими і неорієнтованими графами.

**Основні види графів:**

* **Неорієнтовані графи:** Ребра не мають напрямку. Якщо ребро з'єднує вершини A і B, це означає, що зв'язок є двостороннім (від A до B і від B до A).
    * **Приклад:** Соціальна мережа, де дружба є взаємною.
* **Орієнтовані графи (діграфи):** Ребра мають напрямок. Ребро від A до B означає зв'язок тільки в одному напрямку (від A до B), але не обов'язково від B до A.
    * **Приклад:** Дорожня мережа з вулицями з одностороннім рухом, або посилання на веб-сайтах.
* **Зважені графи:** Кожне ребро має пов'язане з ним числове значення, зване "вагою" (або "вартістю", "довжиною"). Ваги можуть представляти відстань, час, вартість тощо.
    * **Приклад:** Дорожня мережа, де вага ребра — це відстань між містами.
* **Незважені графи:** Ребра не мають ваг. Усі зв'язки вважаються рівнозначними.
* **Циклічні графи:** Містять хоча б один цикл (шлях, що починається і закінчується в одній і тій же вершині, проходячи через інші вершини без повторень ребер).
* **Ациклічні графи:** Не містять жодного циклу.
    * **Орієнтовані ациклічні графи (DAG):** Особливо важливі для моделювання залежностей (наприклад, у плануванні проектів).
* **Зв'язні графи:** Між будь-якими двома вершинами існує шлях.
* **Повні графи:** Кожна пара вершин з'єднана ребром.
* **Мультиграфи:** Можуть мати декілька ребер між однією і тією ж парою вершин, а також петлі (ребро, що з'єднує вершину саму з собою).
* **Прості графи:** Не мають петель і між будь-якою парою вершин є не більше одного ребра.

**Відмінності між орієнтованими і неорієнтованими графами:**

| Характеристика      | Неорієнтований граф                                | Орієнтований граф (Діграф)                                    |
| :------------------ | :------------------------------------------------- | :------------------------------------------------------------ |
| **Ребра** | Не мають напрямку. Позначаються як $\{u, v\}$ або $(u, v)$ | Мають напрямок. Позначаються як $(u, v)$, де $u \to v$     |
| **Зв'язок** | Симетричний: якщо $u$ з'єднаний з $v$, то $v$ з'єднаний з $u$. | Асиметричний: якщо є ребро $(u, v)$, не означає, що є $(v, u)$. |
| **Матриця суміжності** | Симетрична відносно головної діагоналі ($A_{ij} = A_{ji}$) | Несиметрична                                                  |
| **Ступінь вершини** | Ступінь вершини = кількість інцидентних ребер    | Є вхідний ступінь (кількість ребер, що входять) та вихідний ступінь (кількість ребер, що виходять) |

## 3. Як можна представити граф у пам’яті комп'ютера? Опишіть структури даних, які використовуються для зберігання графів.

У пам'яті комп'ютера графи можна представити різними способами, кожен з яких має свої переваги та недоліки залежно від типу графа та операцій, які потрібно виконувати. Найпоширеніші структури даних:

1.  **Матриця суміжності (Adjacency Matrix):**
    * **Опис:** Це двовимірний масив (матриця) розміром $V \times V$, де $V$ — кількість вершин. Елемент $A[i][j]$ матриці дорівнює 1 (або вазі ребра), якщо існує ребро від вершини $i$ до вершини $j$, і 0 (або $\infty$, якщо зважений), якщо ребра немає.
    * **Для неорієнтованих графів:** Матриця буде симетричною ($A[i][j] = A[j][i]$).
    * **Для зважених графів:** Замість 1 зберігається вага ребра.
    * **Переваги:** Швидка перевірка наявності ребра між двома вершинами ($O(1)$). Простота реалізації.
    * **Недоліки:** Висока витрата пам'яті для розріджених графів (де мало ребер), $O(V^2)$ пам'яті. Пошук всіх сусідів вершини займає $O(V)$ часу.
    * **Приклад Python (незважений, неорієнтований):**
        ```python
        # Для графа з 4 вершин: 0-1, 0-2, 1-3
        adj_matrix = [
            [0, 1, 1, 0],  # 0 -> 1, 2
            [1, 0, 0, 1],  # 1 -> 0, 3
            [1, 0, 0, 0],  # 2 -> 0
            [0, 1, 0, 0]   # 3 -> 1
        ]
        ```

2.  **Список суміжності (Adjacency List):**
    * **Опис:** Це масив (або словник), де кожен елемент відповідає вершині. Кожен елемент містить список (або зв'язний список) усіх вершин, суміжних з даною вершиною.
    * **Для зважених графів:** У списку зберігаються пари (сусідня вершина, вага).
    * **Переваги:** Ефективне використання пам'яті для розріджених графів ($O(V+E)$). Швидкий доступ до всіх сусідів вершини ($O(deg(V))$).
    * **Недоліки:** Перевірка наявності ребра між двома вершинами може займати $O(deg(V))$ часу.
    * **Приклад Python (незважений, неорієнтований):**
        ```python
        # Для графа з 4 вершин: 0-1, 0-2, 1-3
        adj_list = {
            0: [1, 2],
            1: [0, 3],
            2: [0],
            3: [1]
        }
        # Для зваженого графа:
        # adj_list_weighted = {
        #     1: [(2, 10), (3, 15), (4, 20)],
        #     2: [(1, 10), (3, 35), (4, 25)],
        #     # ...
        # }
        ```

3.  **Список ребер (Edge List):**
    * **Опис:** Це просто список усіх ребер у графі. Кожне ребро представлено парою (для неорієнтованих) або впорядкованою парою (для орієнтованих) вершин, які воно з'єднує. Для зважених графів додається третій елемент — вага (u, v, weight).
    * **Переваги:** Дуже просто реалізувати. Зручно для алгоритмів, які потребують ітерації по всіх ребрах (наприклад, Крускала).
    * **Недоліки:** Дуже неефективний для пошуку сусідів або перевірки наявності ребра ($O(E)$).
    * **Приклад Python (зважений, неорієнтований):**
        ```python
        edge_list = [
            (1, 2, 10),
            (1, 3, 15),
            (1, 4, 20),
            (2, 3, 35),
            (2, 4, 25),
            (3, 4, 30)
        ]
        ```

Вибір представлення залежить від розміру графа (кількості вершин і ребер) і від того, які операції будуть виконуватися найчастіше. Для щільних графів (багато ребер) матриця суміжності може бути кращою, тоді як для розріджених графів перевагу надають списку суміжності. Список ребер часто використовується як допоміжний формат.

## 4. Як працює алгоритм пошуку в ширину (BFS) на графах? Наведіть приклади ситуацій, де застосовується цей алгоритм.

**Алгоритм пошуку в ширину (BFS - Breadth-First Search)** - це алгоритм обходу графа, який досліджує граф шар за шаром, починаючи з заданої стартової вершини. Він відвідує всіх безпосередніх сусідів стартової вершини, потім всіх їхніх непосередніх сусідів, і так далі. Це робить його ідеальним для пошуку найкоротшого шляху в **незважених** графах.

**Принцип роботи BFS:**

1.  **Ініціалізація:**
    * Створити чергу (`Queue`) і додати до неї стартову вершину.
    * Створити набір (`Set`) `visited` (відвідані вершини) і додати до нього стартову вершину.
2.  **Обхід:**
    * Доки черга не порожня:
        * Видалити вершину `v` з початку черги.
        * Обробити вершину `v` (наприклад, вивести її).
        * Для кожної невідвіданої сусідньої вершини `u` до `v`:
            * Додати `u` до `visited`.
            * Додати `u` до кінця черги.

**Приклад кроків BFS:**

Стартовий вузол: A

1.  Черга: `[A]`, Відвідані: `{A}`
2.  Видалити A. Обробити A.
    Сусіди A: B, C.
    Додати B, C до черги.
    Черга: `[B, C]`, Відвідані: `{A, B, C}`
3.  Видалити B. Обробити B.
    Сусіди B: A (відвіданий), D.
    Додати D до черги.
    Черга: `[C, D]`, Відвідані: `{A, B, C, D}`
4.  Видалити C. Обробити C.
    Сусіди C: A (відвіданий), E.
    Додати E до черги.
    Черга: `[D, E]`, Відвідані: `{A, B, C, D, E}`
5.  Видалити D. Обробити D.
    Сусіди D: B (відвіданий), F.
    Додати F до черги.
    Черга: `[E, F]`, Відвідані: `{A, B, C, D, E, F}`
6.  Видалити E. Обробити E.
    Сусіди E: C (відвіданий).
    Черга: `[F]`, Відвідані: `{A, B, C, D, E, F}`
7.  Видалити F. Обробити F.
    Сусіди F: D (відвіданий).
    Черга: `[]`, Відвідані: `{A, B, C, D, E, F}`

Черга порожня, обхід завершено.

**Приклади ситуацій, де застосовується BFS:**

* **Пошук найкоротшого шляху в незваженому графі:** BFS завжди знаходить найкоротший шлях (у термінах кількості ребер) від стартової вершини до всіх інших.
* **Знаходження всіх компонентів зв'язності графа:** Послідовний запуск BFS з невідвіданих вершин дозволяє знайти всі зв'язні компоненти.
* **Пошук у лабіринтах:** Знаходження найкоротшого шляху від старту до фінішу.
* **Виявлення циклів у неорієнтованих графах:** Якщо під час BFS виявляється ребро, що веде до вже відвіданої вершини, яка не є безпосереднім попередником (батьком) у дереві BFS, то в графі є цикл.
* **Мережеве сканування (наприклад, сканери портів):** Обхід мережевих вузлів.
* **Системи рекомендацій:** Пошук "друзів друзів" або схожих елементів на певній "відстані".

## 5. Що таке алгоритм пошуку в глибину (DFS) на графах? Як він відрізняється від BFS? Дайте приклади задач, де використовується DFS.

**Алгоритм пошуку в глибину (DFS - Depth-First Search)** - це алгоритм обходу графа, який, починаючи з довільної вершини, максимально заглиблюється вздовж кожної гілки, перш ніж повертатися (виконувати відкат) і досліджувати інші гілки.

**Принцип роботи DFS:**

1.  **Ініціалізація:**
    * Вибрати стартову вершину.
    * Створити набір (`Set`) `visited` (відвідані вершини).
2.  **Обхід (рекурсивно або за допомогою стека):**
    * Позначити поточну вершину `v` як відвідану і обробити її.
    * Для кожної невідвіданої сусідньої вершини `u` до `v`:
        * Викликати DFS для `u`.
    * Якщо використовуються ітерації (без рекурсії), використовується стек (`Stack`) замість черги.

**Приклад кроків DFS (рекурсивно):**

Стартовий вузол: A

1.  `dfs(A)`
    * Позначити A як відвідану, обробити A.
    * Сусіди A: B, C.
    * Викликати `dfs(B)`
        * Позначити B як відвідану, обробити B.
        * Сусіди B: A (відвіданий), D.
        * Викликати `dfs(D)`
            * Позначити D як відвідану, обробити D.
            * Сусіди D: B (відвіданий), F.
            * Викликати `dfs(F)`
                * Позначити F як відвідану, обробити F.
                * Сусіди F: D (відвіданий).
                * Повернення з `dfs(F)`.
            * Всі сусіди D відвідані.
            * Повернення з `dfs(D)`.
        * Всі сусіди B відвідані.
        * Повернення з `dfs(B)`.
    * Викликати `dfs(C)` (після повернення з `dfs(B)`)
        * Позначити C як відвідану, обробити C.
        * Сусіди C: A (відвіданий), E.
        * Викликати `dfs(E)`
            * Позначити E як відвідану, обробити E.
            * Сусіди E: C (відвіданий).
            * Повернення з `dfs(E)`.
        * Всі сусіди C відвідані.
        * Повернення з `dfs(C)`.
    * Всі сусіди A відвідані.
    * Повернення з `dfs(A)`.

Обхід завершено.

**Відмінності DFS від BFS:**

| Характеристика       | DFS (Пошук у глибину)                                | BFS (Пошук у ширину)                                    |
| :------------------- | :--------------------------------------------------- | :-------------------------------------------------------- |
| **Структура даних** | Стек (явно або неявно через рекурсію)               | Черга                                                   |
| **Порядок обходу** | Заглиблюється якомога далі по одній гілці, потім повертається. | Обхід "шарами", відвідуючи всі вузли на одному рівні перед переходом до наступного. |
| **Знаходження найкоротшого шляху** | Не гарантує найкоротший шлях (для незважених графів). | Гарантує найкоротший шлях (для незважених графів).     |
| **Використання пам'яті** | Може бути меншим для "глибоких" графів ($O(V)$)       | Може бути більшим для "широких" графів ($O(V)$), оскільки зберігає більше вузлів у черзі одночасно. |
| **Реалізація** | Часто рекурсивна, також можлива ітеративна зі стеком. | Зазвичай ітеративна з чергою.                         |

**Приклади задач, де використовується DFS:**

* **Виявлення циклів в орієнтованих графах:** Якщо під час DFS зустрічається ребро, що веде до вершини, яка вже знаходиться у поточному шляху рекурсії (тобто в стеку викликів), це вказує на цикл.
* **Топологічне сортування:** Для орієнтованих ациклічних графів (DAGs), DFS використовується для упорядкування вершин таким чином, що для кожного орієнтованого ребра $u \to v$, $u$ йде раніше за $v$.
* **Знаходження компонент сильної зв'язності (SCC) в орієнтованих графах:** Використовуються алгоритми, засновані на DFS (наприклад, алгоритм Косарайю або Тарджана).
* **Вирішення головоломок та ігор:** Наприклад, вирішення лабіринтів, пошук шляхів у настільних іграх, де можливе "відкочування" (backtracking).
* **Пошук шляху до цілі:** Якщо шлях існує, DFS його знайде, але не обов'язково найкоротший.
* **Генерація лабіринтів:** DFS може бути використаний для створення лабіринтів.

## 6. Опишіть алгоритм Дейкстри для пошуку найкоротшого шляху в графі. Які умови повинні виконуватися для успішної роботи цього алгоритму?

**Алгоритм Дейкстри** - це жадібний алгоритм, призначений для знаходження найкоротших шляхів від однієї заданої **стартової вершини** до всіх інших вершин у **зваженому графі**. Він ефективний для графів з **невід'ємними вагами** ребер.

**Принцип роботи алгоритму Дейкстри:**

Алгоритм підтримує множину вершин, для яких вже відомий найкоротший шлях від стартової вершини. Він ітераційно вибирає найближчу невідвідану вершину і оновлює відстані до її сусідів.

1.  **Ініціалізація:**
    * Створити словник (або масив) `distances`, де для кожної вершини $v$ зберігається поточна найкоротша відстань від стартової вершини $s$. Спочатку `distances[s] = 0`, а для всіх інших вершин $v \neq s$, `distances[v] = \infty$.
    * Створити пріоритетну чергу (`Priority Queue`), яка зберігає пари `(відстань, вершина)`. Додати до неї `(0, s)`.
    * Створити множину `visited`, щоб відстежувати вершини, для яких вже знайдено остаточний найкоротший шлях.

2.  **Основний цикл:**
    * Поки пріоритетна черга не порожня:
        * Витягнути з черги вершину `u` з найменшою поточною відстанню `d_u`.
        * Якщо `u` вже була відвідана (`u` в `visited`), або `d_u` більша за вже відому `distances[u]` (це може статися через дублікати в пріоритетній черзі), пропустити цю ітерацію.
        * Додати `u` до `visited`.
        * **Оновлення відстаней сусідів (`релаксація`):** Для кожного сусіда `v` вершини `u`:
            * Обчислити нову відстань: `new_distance = distances[u] + weight(u, v)`.
            * Якщо `new_distance < distances[v]`:
                * Оновити `distances[v] = new_distance`.
                * Додати `(new_distance, v)` до пріоритетної черги.

3.  **Завершення:** Після того, як пріоритетна черга стане порожньою, `distances` буде містити найкоротші відстані від стартової вершини до всіх інших вершин.

**Приклад (спрощено):**

Стартова вершина: A

1.  `dist = {A:0, B:inf, C:inf, D:inf}`. Черга: `[(0,A)]`. Visited: `{}`.
2.  Витягти `(0,A)`. Додати A до `visited`.
    Сусіди A: B (вага 10), C (вага 15).
    `dist[B] = min(inf, 0+10) = 10`. Додати `(10,B)` до черги.
    `dist[C] = min(inf, 0+15) = 15`. Додати `(15,C)` до черги.
    Черга: `[(10,B), (15,C)]`.
3.  Витягти `(10,B)`. Додати B до `visited`.
    Сусіди B: D (вага 25).
    `dist[D] = min(inf, 10+25) = 35`. Додати `(35,D)` до черги.
    Черга: `[(15,C), (35,D)]`.
4.  Витягти `(15,C)`. Додати C до `visited`.
    Сусіди C: D (вага 30).
    `new_distance = dist[C] + weight(C,D) = 15 + 30 = 45`.
    `dist[D] = min(35, 45) = 35`. (Не оновлюємо, бо 35 менше).
    Черга: `[(35,D)]`.
5.  Витягти `(35,D)`. Додати D до `visited`.
    Сусіди D: немає невідвіданих.
    Черга: `[]`.

Результат: `dist = {A:0, B:10, C:15, D:35}` (шлях A->B->D для D).

**Умови для успішної роботи алгоритму Дейкстри:**

1.  **Невід'ємні ваги ребер:** Це найважливіша умова. Алгоритм Дейкстри не працює коректно, якщо граф містить ребра з від'ємними вагами. Якщо є від'ємні ваги, то потрібно використовувати інші алгоритми, такі як алгоритм Беллмана-Форда або алгоритм Флойда-Уоршелла. Причина в тому, що Дейкстра припускає, що після того, як вершина додана до `visited` (її відстань вважається остаточною), її відстань більше не зміниться, що не так при від'ємних вагах.
2.  **Відсутність від'ємних циклів:** Навіть якщо всі ваги ребер невід'ємні, але існує від'ємний цикл, алгоритм Дейкстри не визначить найкоротшого шляху (оскільки шлях може ставати нескінченно малим, проходячи через від'ємний цикл). Однак, оскільки Дейкстра не працює з від'ємними вагами взагалі, ця умова є наслідком першої.

**Додаткові умови (не є вимогами для коректності, але впливають на ефективність):**

* **Граф може бути орієнтованим або неорієнтованим:** Для неорієнтованого графа ребро $(u, v)$ розглядається як два орієнтовані ребра $(u \to v)$ і $(v \to u)$ з однаковою вагою.
* **Граф може бути зв'язним або незв'язним:** Якщо граф незв'язний і цільова вершина знаходиться в іншій компоненті зв'язності, її відстань залишиться $\infty$.